# Myocardial Infraction Complications Analysis

## 0. Introduction

In [76]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [77]:
data = pd.read_csv("MI.data", header=None)
data.columns = ["ID", "AGE", "SEX", "INF_ANAM", "STENOK_AN", "FK_STENOK", "IBS_POST", "IBS_NASL", "GB", "SIM_GIPERT", "DLIT_AG", "ZSN_A", "nr_11", "nr_01", "nr_02", "nr_03", "nr_04", "nr_07", "nr_08", "np_01", "np_04", "np_05", "np_07", "np_08", "np_09", "np_10", "endocr_01", "endocr_02", "endocr_03", "zab_leg_01", "zab_leg_02", "zab_leg_03", "zab_leg_04", "zab_leg_06", "S_AD_KBRIG", "D_AD_KBRIG", "S_AD_ORIT", "D_AD_ORIT", "O_L_POST", "K_SH_POST", "MP_TP_POST", "SVT_POST", "GT_POST", "FIB_G_POST", "ant_im", "lat_im", "inf_im", "post_im", "IM_PG_P", "ritm_ecg_p_01", "ritm_ecg_p_02", "ritm_ecg_p_04", "ritm_ecg_p_06", "ritm_ecg_p_07", "ritm_ecg_p_08", "n_r_ecg_p_01", "n_r_ecg_p_02", "n_r_ecg_p_03", "n_r_ecg_p_04", "n_r_ecg_p_05", "n_r_ecg_p_06", "n_r_ecg_p_08", "n_r_ecg_p_09", "n_r_ecg_p_10", "n_p_ecg_p_01", "n_p_ecg_p_03", "n_p_ecg_p_04", "n_p_ecg_p_05", "n_p_ecg_p_06", "n_p_ecg_p_07", "n_p_ecg_p_08", "n_p_ecg_p_09", "n_p_ecg_p_10", "n_p_ecg_p_11", "n_p_ecg_p_12", "fibr_ter_01", "fibr_ter_02", "fibr_ter_03", "fibr_ter_05", "fibr_ter_06", "fibr_ter_07", "fibr_ter_08", "GIPO_K", "K_BLOOD", "GIPER_NA", "NA_BLOOD", "ALT_BLOOD", "AST_BLOOD", "KFK_BLOOD", "L_BLOOD", "ROE", "TIME_B_S", "R_AB_1_n", "R_AB_2_n", "R_AB_3_n", "NA_KB", "NOT_NA_KB", "LID_KB", "NITR_S", "NA_R_1_n", "NA_R_2_n", "NA_R_3_n", "NOT_NA_1_n", "NOT_NA_2_n", "NOT_NA_3_n", "LID_S_n", "B_BLOK_S_n", "ANT_CA_S_n", "GEPAR_S_n", "ASP_S_n", "TIKL_S_n", "TRENT_S_n", "FIBR_PREDS", "PREDS_TAH", "JELUD_TAH", "FIBR_JELUD", "A_V_BLOK", "OTEK_LANC", "RAZRIV", "DRESSLER", "ZSN", "REC_IM", "P_IM_STEN", "LET_IS"]
data.replace("?", np.NaN, inplace=True)
data = data.apply(pd.to_numeric, errors = "coerce")
data

,ID,AGE,SEX,INF_ANAM,STENOK_AN,FK_STENOK,IBS_POST,IBS_NASL,GB,SIM_GIPERT,...,JELUD_TAH,FIBR_JELUD,A_V_BLOK,OTEK_LANC,RAZRIV,DRESSLER,ZSN,REC_IM,P_IM_STEN,LET_IS
0,1,77.0,1,2.0,1.0,1.0,2.0,NaN,3.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,2,55.0,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,3,52.0,1,0.0,0.0,0.0,2.0,NaN,2.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,4,68.0,0,0.0,0.0,0.0,2.0,NaN,2.0,0.0,...,0,0,0,0,0,0,1,0,0,0
4,5,60.0,1,0.0,0.0,0.0,2.0,NaN,3.0,0.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1695,1696,77.0,0,0.0,4.0,2.0,1.0,NaN,2.0,0.0,...,0,0,1,0,1,0,0,0,0,3
1696,1697,70.0,0,0.0,6.0,2.0,1.0,NaN,2.0,0.0,...,0,0,0,0,0,0,0,0,0,1
1697,1698,55.0,1,3.0,6.0,2.0,2.0,NaN,0.0,0.0,...,0,0,0,0,0,0,0,1,0,6
1698,1699,79.0,0,2.0,2.0,2.0,1.0,NaN,2.0,0.0,...,0,0,0,1,0,0,0,0,0,1


In [78]:
related_time_features = ["R_AB_1_n","R_AB_2_n", "R_AB_3_n", "NA_R_1_n", "NA_R_2_n", "NA_R_3_n", "NOT_NA_1_n","NOT_NA_2_n", "NOT_NA_3_n"]

## 1. Train-Validation-Test Split

In [79]:
from sklearn.model_selection import train_test_split
X = data.iloc[:, 1:112]
X = X.drop(related_time_features, axis=1)
y = [data["ZSN"], data["FIBR_PREDS"], data["P_IM_STEN"], data["REC_IM"], data["OTEK_LANC"]]

X_train, X_test, y_train, y_test = [], [], [], []

# ! Caution
# i = 0 => ZSN or Chronic heart failure
# i = 1 => FIBR_PREDS or Atrial fibrillation
# i = 2 => P_IM_STEN or Post-infarction angina
# i = 3 => REC_IM or Relapse of the myocardial infarction
# i = 4 => OTEK_LANC or Pulmonary edema

for i in range(len(y)):
    # Separate train/test split for each target variable
    # Split the data into train and temporary sets
    X_train_i, X_test_i, y_train_i, y_test_i = train_test_split(X, y[i], train_size=0.7, random_state=0, stratify=y[i])
    
    X_train.append(X_train_i)
    X_test.append(X_test_i)
    y_train.append(y_train_i)
    y_test.append(y_test_i)


In [80]:
# Check the number of samples in each set and print as a table
results = {"Set": ["Train", "Test"]}
for i in range(len(y)):
    results[f"y{i}"] = [len(y_train[i]), len(y_test[i])]
results = pd.DataFrame(results)
results

,Set,y0,y1,y2,y3,y4
0,Train,1190,1190,1190,1190,1190
1,Test,510,510,510,510,510


## 2. EDA

### Data Overview

In [81]:
data

,ID,AGE,SEX,INF_ANAM,STENOK_AN,FK_STENOK,IBS_POST,IBS_NASL,GB,SIM_GIPERT,...,JELUD_TAH,FIBR_JELUD,A_V_BLOK,OTEK_LANC,RAZRIV,DRESSLER,ZSN,REC_IM,P_IM_STEN,LET_IS
0,1,77.0,1,2.0,1.0,1.0,2.0,NaN,3.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,2,55.0,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,3,52.0,1,0.0,0.0,0.0,2.0,NaN,2.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,4,68.0,0,0.0,0.0,0.0,2.0,NaN,2.0,0.0,...,0,0,0,0,0,0,1,0,0,0
4,5,60.0,1,0.0,0.0,0.0,2.0,NaN,3.0,0.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1695,1696,77.0,0,0.0,4.0,2.0,1.0,NaN,2.0,0.0,...,0,0,1,0,1,0,0,0,0,3
1696,1697,70.0,0,0.0,6.0,2.0,1.0,NaN,2.0,0.0,...,0,0,0,0,0,0,0,0,0,1
1697,1698,55.0,1,3.0,6.0,2.0,2.0,NaN,0.0,0.0,...,0,0,0,0,0,0,0,1,0,6
1698,1699,79.0,0,2.0,2.0,2.0,1.0,NaN,2.0,0.0,...,0,0,0,1,0,0,0,0,0,1


### Target Balance Check

In [82]:
# The number of instances in the dataset
number_of_instances = len(data)
# Create a table to show the balance of each target variable
results = {"Target": [], "0": [], "1": []}
for i in range(len(y)):
    results["Target"].append(f"y{i}")
    results["0"].append(y[i].value_counts()[0] / number_of_instances)
    results["1"].append(y[i].value_counts()[1] / number_of_instances)
results = pd.DataFrame(results)
results

,Target,0,1
0,y0,0.768235,0.231765
1,y1,0.900000,0.100000
2,y2,0.912941,0.087059
3,y3,0.906471,0.093529
4,y4,0.906471,0.093529


### Check for missing values

In [83]:
# Create a table representing number of missing values of each feature, sort them descendingly
missing_values = X.isnull().sum().sort_values(ascending=False)
missing_values = missing_values[missing_values > 0]
missing_values = pd.DataFrame(missing_values, columns=["Number of Missing Values"])
missing_values


,Number of Missing Values
KFK_BLOOD,1696
IBS_NASL,1628
S_AD_KBRIG,1076
D_AD_KBRIG,1076
NOT_NA_KB,686
...,...
zab_leg_02,7
zab_leg_03,7
zab_leg_04,7
INF_ANAM,4


### Data Information of Feature

In [84]:
X.describe()

,AGE,SEX,INF_ANAM,STENOK_AN,FK_STENOK,IBS_POST,IBS_NASL,GB,SIM_GIPERT,DLIT_AG,...,NOT_NA_KB,LID_KB,NITR_S,LID_S_n,B_BLOK_S_n,ANT_CA_S_n,GEPAR_S_n,ASP_S_n,TIKL_S_n,TRENT_S_n
count,1692.000000,1700.000000,1696.000000,1594.000000,1627.000000,1649.000000,72.00000,1691.000000,1692.000000,1452.000000,...,1014.000000,1023.000000,1691.000000,1690.000000,1689.000000,1687.000000,1683.000000,1683.000000,1684.000000,1684.000000
mean,61.856974,0.626471,0.554835,2.316186,1.205286,1.160703,0.37500,1.393258,0.033688,3.340220,...,0.691321,0.387097,0.115316,0.283432,0.127294,0.666864,0.714795,0.743910,0.017815,0.202494
std,11.259936,0.483883,0.836801,2.440586,1.040814,0.801400,0.48752,1.088803,0.180478,3.098646,...,0.462176,0.487324,0.319498,0.450798,0.333401,0.471474,0.451646,0.436602,0.132317,0.401978
min,26.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,54.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,63.000000,1.000000,0.000000,1.000000,2.000000,1.000000,0.00000,2.000000,0.000000,3.000000,...,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000
75%,70.000000,1.000000,1.000000,5.000000,2.000000,2.000000,1.00000,2.000000,0.000000,7.000000,...,1.000000,1.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000
max,92.000000,1.000000,3.000000,6.000000,4.000000,2.000000,1.00000,3.000000,1.000000,7.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [85]:
continuous_features = ["AGE", "S_AD_ORIT", "D_AD_ORIT", "K_BLOOD", "NA_BLOOD", "ALT_BLOOD", "AST_BLOOD", "L_BLOOD", "ROE"]
binary_categorical_features = [x for x in X.columns if x not in continuous_features]

In [86]:
binary_features = []
for col in binary_categorical_features:
    if  X[col].max() == 1:
        binary_features.append(col)

In [87]:
categorical_features = [x for x in binary_categorical_features if x not in binary_features]

## 3. Preprocessing

In [88]:
import copy
unprocessed_data = X_train
%store unprocessed_data
unprocessed_data[0]

Stored 'unprocessed_data' (list)


,AGE,SEX,INF_ANAM,STENOK_AN,FK_STENOK,IBS_POST,IBS_NASL,GB,SIM_GIPERT,DLIT_AG,...,NOT_NA_KB,LID_KB,NITR_S,LID_S_n,B_BLOK_S_n,ANT_CA_S_n,GEPAR_S_n,ASP_S_n,TIKL_S_n,TRENT_S_n
1458,66.0,1,0.0,6.0,2.0,2.0,NaN,2.0,0.0,7.0,...,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0
497,56.0,1,1.0,0.0,0.0,0.0,NaN,2.0,0.0,NaN,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
793,64.0,1,0.0,NaN,2.0,1.0,NaN,2.0,0.0,NaN,...,NaN,NaN,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
595,70.0,1,0.0,6.0,2.0,1.0,0.0,0.0,0.0,0.0,...,NaN,NaN,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0
1004,64.0,0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1261,74.0,0,0.0,NaN,2.0,1.0,NaN,0.0,0.0,0.0,...,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1311,63.0,1,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
317,63.0,1,0.0,6.0,2.0,1.0,NaN,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
24,67.0,1,1.0,6.0,2.0,1.0,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0


In [89]:
from sklearn.base import BaseEstimator, TransformerMixin
import pandas as pd

class ColumnDropper(BaseEstimator, TransformerMixin):
    def __init__(self, threshold):
        self.columns_to_drop = None
        self.threshold = threshold

    def fit(self, X, y=None):
        X = pd.DataFrame(X)
        self.columns_to_drop = X.columns[X.isnull().sum() > self.threshold]
        return self

    def transform(self, X):
        # Ensure the input is a DataFrame
        X = pd.DataFrame(X)
        global X_keep 
        X_keep = list(X.columns[~X.columns.isin(self.columns_to_drop)])
        return X.drop(columns=self.columns_to_drop)
    

class RowDropper(BaseEstimator, TransformerMixin):
    def __init__(self, threshold):
        self.rows_to_drop = None
        self.threshold = threshold

    def fit(self, X, y=None):
        self.rows_to_drop = X.index[X.isnull().sum(axis=1) > self.threshold]
        return self

    def transform(self, X):
        # Ensure the input is a DataFrame
        X = pd.DataFrame(X)
        return X.drop(index=self.rows_to_drop)
    

In [90]:
# Define outliers handler class
class OutliersHandler(BaseEstimator, TransformerMixin):
    def __init__(self, coefficient):
        self.coefficient = coefficient
        self.lower_bounds = None
        self.upper_bounds = None

    def fit(self, X, y=None):
        X = pd.DataFrame(X)
        q1 = X.quantile(0.25)
        q3 = X.quantile(0.75)
        iqr = q3 - q1
        self.lower_bounds = q1 - iqr * self.coefficient
        self.upper_bounds = q3 + iqr * self.coefficient
        return self

    def transform(self, X):
        # Ensure the input is a DataFrame
        X = pd.DataFrame(X)
        for feature in X.columns:
            if feature in continuous_features: # if feature is continuous, because clipping in the other one will probably result in missing information
                X[feature] = X[feature].clip(self.lower_bounds[feature], self.upper_bounds[feature])
        return X

In [91]:
def impute_by_type(X, continuous_features=None, categorical_features=None, binary_features=None, binary_categorical_features=None):
    """Fills missing values based on data type, handling potential errors, works with subsets"""
    if categorical_features is not None:
        for feature in categorical_features:
            if feature in X.columns:
                try:
                    X[feature].fillna(X[feature].mode()[0], inplace=True)
                except KeyError:
                    pass
    if continuous_features is not None:
        for feature in continuous_features:
            if feature in X.columns:
                try:
                    X[feature].fillna(X[feature].mean(axis=0), inplace=True)
                except KeyError:
                    pass
    if binary_features is not None:
        for feature in binary_features:
            if feature in X.columns:
                try:
                    X[feature].fillna(X[feature].mode()[0], inplace=True)
                except KeyError:
                    pass
    if binary_categorical_features is not None and binary_features is None and categorical_features is None:
        for feature in binary_categorical_features:
            if feature in X.columns:
                try:
                    X[feature].fillna(X[feature].mode()[0], inplace=True)
                except KeyError:
                    pass
    return X

In [92]:
# from sklearn.impute import KNNImputer
# def impute_by_type(X, continuous_features=None, categorical_features=None, binary_features=None):
#     """Fills missing values based on data type, handling potential errors, works with subsets"""
#     if categorical_features is not None:
#         for feature in categorical_features:
#             if feature in X.columns:
#                 try:
#                     X[feature].fillna(X[feature].mode()[0], inplace=True)
#                 except KeyError:
#                     pass
# 
#     imputer = KNNImputer(n_neighbors=10)
#     imputer.fit(X)
#     X_imputed = pd.DataFrame(imputer.transform(X), columns=X.columns, index=X.index)
#     if continuous_features is not None:
#         for feature in continuous_features:
#             if feature in X.columns:
#                 try:
#                     temp_df = pd.DataFrame({feature: X_imputed[feature]})
#                     X.update(temp_df)
#                     # print(X[feature].isnull().sum(), X_imputed[feature].isnull().sum())
#                 except KeyError:
#                     pass
#     if binary_features is not None:
#         for feature in binary_features:
#             if feature in X.columns:
#                 try:
#                     temp_df = pd.DataFrame({feature: X_imputed[feature]})
#                     X.update(temp_df)
#                     # print(X[feature].isnull().sum(), X_imputed[feature].isnull().sum())
#                 except KeyError:
#                     pass
#     return X

In [93]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer

# Define dropper pipeline
dropper = Pipeline(steps=[
    ('column_dropper', ColumnDropper(threshold=100)),
    ('row_dropper', RowDropper(threshold=100))
])

# Define preprocessing pipeline
preprocessing_pipeline = Pipeline(steps=[
    ('dropper', dropper),
    ('outliers_clipper',OutliersHandler(coefficient=1.5)),
    ('imputation', FunctionTransformer(impute_by_type, kw_args={"continuous_features":continuous_features, "binary_features": binary_features, "categorical_features": categorical_features })),  # Fill missing values using mean/mode
    ('scaling', StandardScaler())  # Standardize features by removing the mean and scaling to unit variance
])

# Apply the preprocessing pipeline to each set
for i in range(len(y)):
    X_train[i] = preprocessing_pipeline.fit_transform(X_train[i])
    X_test[i] = preprocessing_pipeline.transform(X_test[i])

preprocessed_data = pd.DataFrame(data=X_train[0], columns=X_keep)
%store preprocessed_data
preprocessed_data

Stored 'preprocessed_data' (DataFrame)


,AGE,SEX,INF_ANAM,STENOK_AN,FK_STENOK,IBS_POST,GB,SIM_GIPERT,ZSN_A,nr_11,...,L_BLOOD,TIME_B_S,NITR_S,LID_S_n,B_BLOK_S_n,ANT_CA_S_n,GEPAR_S_n,ASP_S_n,TIKL_S_n,TRENT_S_n
0,0.340030,0.785397,-0.661077,1.595975,0.743926,1.015624,0.564308,-0.193621,-0.278129,-0.171499,...,1.270216,0.537529,2.653402,-0.637667,-0.373961,0.700877,0.618154,-1.684847,-0.137243,2.005267
1,-0.566563,0.785397,0.560478,-0.888818,-1.203867,-1.491829,0.564308,-0.193621,-0.278129,-0.171499,...,-1.371753,1.253432,-0.376875,-0.637667,-0.373961,-1.426785,0.618154,0.593526,-0.137243,-0.498687
2,0.158711,0.785397,-0.661077,-0.888818,0.743926,-0.238103,0.564308,-0.193621,-0.278129,-0.171499,...,1.036118,-0.178374,-0.376875,-0.637667,-0.373961,-1.426785,0.618154,0.593526,-0.137243,2.005267
3,0.702667,0.785397,-0.661077,1.595975,0.743926,-0.238103,-1.278015,-0.193621,-0.278129,-0.171499,...,-0.335031,1.611383,-0.376875,-0.637667,-0.373961,0.700877,0.618154,-1.684847,-0.137243,2.005267
4,0.158711,-1.273242,-0.661077,-0.888818,-1.203867,1.015624,0.564308,-0.193621,1.261282,-0.171499,...,-0.535687,1.253432,-0.376875,-0.637667,-0.373961,-1.426785,0.618154,-1.684847,-0.137243,-0.498687
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1185,1.065304,-1.273242,-0.661077,-0.888818,0.743926,-0.238103,-1.278015,-0.193621,-0.278129,-0.171499,...,2.340381,-0.536326,-0.376875,-0.637667,-0.373961,-1.426785,-1.617721,-1.684847,-0.137243,2.005267
1186,0.068052,0.785397,-0.661077,-0.888818,-1.203867,-1.491829,-1.278015,-0.193621,-0.278129,-0.171499,...,-0.335031,1.253432,-0.376875,-0.637667,-0.373961,-1.426785,0.618154,0.593526,-0.137243,-0.498687
1187,0.068052,0.785397,-0.661077,1.595975,0.743926,-0.238103,-1.278015,-0.193621,1.261282,-0.171499,...,0.868904,-0.894277,-0.376875,-0.637667,-0.373961,-1.426785,0.618154,0.593526,-0.137243,2.005267
1188,0.430689,0.785397,0.560478,1.595975,0.743926,-0.238103,-1.278015,-0.193621,1.261282,-0.171499,...,0.501035,-0.894277,-0.376875,-0.637667,-0.373961,0.700877,-1.617721,0.593526,-0.137243,-0.498687


## 4. Model Selection

In [94]:
#create a pipeline to predict y using svm
from sklearn.svm import SVC

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score

# Define the pipeline
model_pipeline = Pipeline(steps=[
    ('model', SVC())
])

# Define the hyperparameters grid
param_grid= {
    'model__C': [0.1, 1], 
    'model__gamma': [1, 0.1], 
    'model__kernel': ['rbf', 'linear']
}

# Define the grid search
grid_search = GridSearchCV(model_pipeline, param_grid, scoring = 'roc_auc', cv=5, n_jobs=-1)

# Define a list to store the best parameters
best_params = []
best_models = []

# Fit the grid search for each target variable
for i in range(len(y)):
    grid_search.fit(X_train[i], y_train[i])
    # Print the best parameters
    print("-----------------------------------------------------------------------")
    print(f"Best parameters for y{i}: {grid_search.best_params_}")
    # Store the parameters
    best_params.append(grid_search.best_params_)
    best_models.append(grid_search.best_estimator_)

    

-----------------------------------------------------------------------
Best parameters for y0: {'model__C': 1, 'model__gamma': 0.1, 'model__kernel': 'rbf'}
-----------------------------------------------------------------------
Best parameters for y1: {'model__C': 0.1, 'model__gamma': 0.1, 'model__kernel': 'rbf'}
-----------------------------------------------------------------------
Best parameters for y2: {'model__C': 1, 'model__gamma': 1, 'model__kernel': 'linear'}
-----------------------------------------------------------------------
Best parameters for y3: {'model__C': 0.1, 'model__gamma': 0.1, 'model__kernel': 'rbf'}
-----------------------------------------------------------------------
Best parameters for y4: {'model__C': 0.1, 'model__gamma': 0.1, 'model__kernel': 'rbf'}


## 5. Model Evaluation

In [99]:
# Create 5 tables for evaluating 5 targets, each table contains 1 column representing the model, 5 rows representing the metrics, which are accuracy, precision, recall, f1-score, and ROC AUC
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

for i in range(len(y)):
    results = {"Metric" : ["Accuracy", "Precision", "Recall", "F1", "ROC AUC"]}
    # # Set the best parameters for the model in the pipeline
    # model_pipeline.set_params(**best_params[i])
    # 
    # # Fit the model with the best parameters
    # model_pipeline.fit(X_train[i], y_train[i])

    # Predict the target variable
    y_pred = best_models[i].predict(X_test[i])

    # Calculate the metrics
    # accuracy = accuracy_score(y_test[i], y_pred)
    # precision = precision_score(y_test[i], y_pred)
    # recall = recall_score(y_test[i], y_pred)
    # f1 = f1_score(y_test[i], y_pred)
    # roc_auc = roc_auc_score(y_test[i], y_pred)
    report = classification_report(y_test[i], y_pred, output_dict=False)
    # Store the metrics
    # results["SVM"] = [accuracy, precision, recall, f1, roc_auc]
    # results = pd.DataFrame(results)
    
    print("-------------------------------------------------")
    if i==0:
        print("Result for ZSN or Chronic heart failure")
    if i==1:
        print("Result for FIBR_PREDS or Atrial fibrillation")
    if i==2:
        print("Result for P_IM_STEN or Post-infarction angina")
    if i==3:
        print("Result for REC_IM or Relapse of the myocardial infarction")
    if i==4:
        print("Result for OTEK_LANC or Pulmonary edema")
    # print(results)
    # del results
    print(report)
    


-------------------------------------------------
Result for ZSN or Chronic heart failure
              precision    recall  f1-score   support

           0       0.77      1.00      0.87       392
           1       0.00      0.00      0.00       118

    accuracy                           0.77       510
   macro avg       0.38      0.50      0.43       510
weighted avg       0.59      0.77      0.67       510

-------------------------------------------------
Result for FIBR_PREDS or Atrial fibrillation
              precision    recall  f1-score   support

           0       0.90      1.00      0.95       459
           1       0.00      0.00      0.00        51

    accuracy                           0.90       510
   macro avg       0.45      0.50      0.47       510
weighted avg       0.81      0.90      0.85       510

-------------------------------------------------
Result for P_IM_STEN or Post-infarction angina
              precision    recall  f1-score   support

         